27/01 Alex's code calculates the energy of the system initially, performs the swap move,
calculates the new energy of the system, then does MH algorithm to decide whether to stay or go back
this seems wildly inefficient.

Currently evaluates the energy of the entire lattice twice per iteration

Ising hamiltonian:
$$ H = -J \sum^N_{\langle i,j \rangle} S_i S_j$$
$$ S_i,S_j \in { -1,1 }$$

For site S(x,y), evaluating nearest neighbours:
$$H(x,y) = - J S_{x,y}[S_{x+1,y} + S_{x-1,y}+ S_{x,y+1}+S_{x,y-1}]$$

So summing 4 nearest neighbours for each lattice site and multiplying by site value gives each sites contribution to the energy.
We can do this using np.roll(Python) or circshift(Julia)

Sort of vectorised so more efficient (but still evaluating the energy of sites that haven't been affected by the move)


In Alex's code candidate_generating_function! is determined in run Metroplois algorithm function, deciding whether to do random swap move or slice rotation

TO DO:
Build correlation function - DONE
- why does Alex's code have two functions that do the same thing


"Generating function"
no need for inverse as each flip is binary
(assuming we don't ever produce moves that are non-commutative ie series of flips)

Autocorrelation function:

$$C(t)  = \frac{1}{N^2} \sum^{N^2}_i \delta_{\sigma_i(t), \sigma_i(0)}$$

Reversing functions successfully implemented. Will need to ensure all flip functions take the same arguments:
- reverse argument
- reversing information so if reverse true we can flip back

29/01/25
After lots of debugging I have a script that runs basic MH algorithm for various beta. Seems to converge.
Now to mess around with values to get some nice hysteresis?<br>

TO DO in future: Use previous magnetisation as starting point for next iteration?

- Additional functions added - haven't tested them out yet. Need to add manual input of which swapping function to use.
- Main challenge next is working out the workflow in terms of what data needs to be generated - great we can monte-carlo anneal but what are the interesting parameters?

30/01/25 Fig1 is average energy not magnetisation which is a bit more interesting. Adjust for graphing 1/beta ~ temperature.

Use autocorrelation run to determine number of iterations required to be uncorrelated from previous state
use these n values for subsequenet "measurements" of energy. each measurement being a series of iterations of MC.


series of temperatures logarithmically spaced

first iteration calculated n number of iterations to decorrelate between each temperature value.
once you've got these n values:

for each run cool from high temperature to low temperature, run for n values and then compute the energy
repeat this run for ~50 different runs
so we have energy as a function of temperature 50 times


profile view

10x10 initially
50 copies
100 temperature intevrals
cutoff iterations of 5000

change energy calculation

01/01/25 Realised current energy calculation doesn't generate energy difference of monte carlo move, just the absolute energy.
Also issue with calculating the energy of the entire lattice.
<br>
currently we have function that just calculates energy of lattice
and we apply move function, calculate new energy and use monte carlo to decide whether to go revert this move

however this involves calculating the energy of the entire lattice (twice) per move
instead we should just evaluate the energies that change
<br>
if we have a set of length k of flip sites
$$ { (x_1,y_1),...(x_k,y_k)}$$
then the change in energy from applying the flips is the change in energy of these sites, plus the change in energy of the neighbouring sites.
<br>
These are the same energy (maybe have been double counting before anyway?) so we just need to evaluate the "on-site energies"
<br>
Also as we're just flipping the sign of the site, the change in energy is just twice the original energy
<br>
ie if we have spin in +1 state, and has neighbours (+1,-1,+1,+1)
initial energy is  +1 * (+1 - 1 + 1 + 1) = +2
final energy is -1 * (+1 - 1 + 1 + 1) = -2


### Proposed new structure of flipping

- one single function that decides what flip move to do and generates list of flip sites

- one single function to calculate energy change of flipping
- - mask interactions between flips by setting sites to 0

- one function to do the flipping



Need to work out result of two flipping sites being adjacent whether energy change cancels out?

### In run_monte_carlo_for_beta, the same lattice object is used repeatedly accross copies of the simulation, without resetting, TO FIX

resetting is a bit more nuanced than I originally thought. solved config is obviously minimal energy configuration - might be a bit weird in terms of accessing state you wouldn't be able to access initially

similarly taking "avg" magnetisation sample isn't necessarily useful

best to start by evolving at very high temperature, maybe let beta = 0

PLAN for next session:
- implement function that prepares lattice by evolving at beta = 0
- implement the algorithm discussed in meeting:
- - start at high temperature
- - iterate until decorrelation
- - take energy measurement
- - decrease temperature slightly, repeat
- - once got energy vs temperature set, repeat again, take averages of energies

- implement the calculating decorrelation n algorithm
- -  on first temperature run, calculate the number of iteration it takes to decorrelate at each temperature
- - now when calculating energies for all the copies, no need to run decorrelation function every time


02/02/25
$$ T_i = T_0 \left[(\frac{T_N}{T_0})^{1/N}  \right] ^i $$

Interesting stuff happens at low temperatures, (high betas) so we want more values to be centered on this
$$ \beta_i = \beta_N \left[(\frac{\beta_0}{\beta_N})^{1/N}  \right] ^i


# What does (converged, current_configuration_correlation_funciton value, current_iteration, accepted_candidates) do?


## Have I got the correct energy scaling?

03/02/25
Set up working algorithm as discussed in meeting, with non-linear temperature distribution, generating measurements as cooling and then repeated to get average energies at each temperature. Still unsure about normalisationg factor for the calculating energy function. Also issue with the normalised calculated energy - surely we'd expect average energy of 0?<br>

TO DO: implement method where decorrelation ns are generated first round.

Play around with temperature range for decorrelation n.
May be the case that we can't get a useful n decorrelation


Sanity check - single spin flip, show that energy has smooth second order phase transition
shouldn't see n 

TO DO: 
- energy temperature curve for single spin flips, maybe check different sizes 
- same curve for k-chain, k-line
- reorganise code so decorrelation.jl is included in methods of lattice.jl or montecarlo.jl

next steps:
 - autocorrelation function against time
 ie cool to temperature, then start autocorrelation measurement (iterate monte carlo steps)
 will require many runs, then fit to stretched exponentials
 - plot decay times against temperature
<br>
 then:
 - obtain connectivity by exploring the possible moves of a configuration

06/02/2025
- Realised I was cooling from cold to hot. I got T_start and T_end the wrong way. replaced with T_cold and T_hot to avoid confusion.
- TO FIX: realised flipping the same spin multiple times doesn't make sense as you're still using the energy of the unflipped lattice.
<br>
- prepared files to generate energy curves
- got working correlation curves
- - upping N = 100 gives hugely smoother curve
<br>
- Fitting relaxation curves to VTM, should I take average of runs then fit or fit then take average?

07/02/2025
 - Fitting k line flip gives error of exponeniating complex parameter. Not sure we want a complex parameter fit? Tried running it again and no such issue
 - Julia keeps throwing error about filename global/local variable, should probably fix

 - Next steps: Get working functions for exploring saddle points
 - Need to work out how saddle index density is determined
 - spike in low beta suggests initially prepared incorrectly in energy anneal

## 10/02/25 Meeting summary
Compare single flip ising model to literature - specific heat, susceptibility (fluctuation dissipation theorem)

measure n decorrelation for k = 1, then use these n values for k larger than 1.
find some k' > 1

rescale autocorrelation iteration for every monte-carlo step - so every N = LxL
10,000 monte-carlo steps
will want larger N for lower noise on correlation stuff

new fit A = 1, C = 0

plot log log C
could try fitting log directly

approach low energies using k = 1, then explore saddle index using k moves

don't allow for multiple picks of the same site ie flip and unflips


11/02/25

- Single flips - do single copy runs, calculate variance in energy for each temperature, use fluctuation dissipation theorem.
- Also take the average of all single copy runs, calculate dE/dT to give heat capacity
- $$\Delta S(0\rightarrow T) = \int^T_0 \frac{C(T^{'})}{T^{'}}dT^{'}$$

- When calculating derivative, not sure if better to differentiate and then take average or vice versa?
- fluctuation and derivative results only seem to match when fluctuations multiplied by 100?
- This is due to taking the average after differentiating ^
- Still need to decide whether to take average first, or differentiate first and remember to correct by factor of m copies

12/02/25
- Ran code overnight and still no sign of finishing - serious need of optimising
- Also pressing is reframe number of "Monte Carlo iterations" as N = L^2 attempted moves
- energy_change using deepcopy not ideal as extra memory allocation?
- bitarray for lattice allocation? - probably not memory isn't an issue, more running time
